In [366]:
import numpy as np
import time as T
from PIL import Image
from cresize import cresizeANN, cresizeABL
FBIT = 13
VBIT = 1 << FBIT
HBIT = 1 << (FBIT - 1)

SBIT = 13
vSBIT = 1 << SBIT
hSBIT = 1 << (SBIT - 1)

CR = lambda x,y: ((x << FBIT)//y)
SCR = lambda x,y: ((x << SBIT)//y)
SDR = lambda x: (x + hSBIT) >> SBIT

XINV = np.array([[-1.,  3., -2.,  0.],
                 [ 3., -6., -3.,  6.],
                 [-3.,  3.,  6.,  0.],
                 [ 1.,  0., -1.,  0.]])/6
YINV = XINV.T

def clip(x, up, low):
    x = up if (x > up) else x
    x = low if (x < low) else x
    return x

def reflect(x, up, low):
    x = -x if (x < low) else x
    x = (up+up-x) if (x > up) else x
    return x


In [367]:
r = SCR(3840,100)
w = 80
wr = w * r

In [368]:
SDR(SDR(wr * wr))

9437136

In [369]:
(80 * 3840/100 ) ** 2

9437184.0

In [353]:
b = 4/7
b*b

0.32653061224489793

In [351]:
42 / (1 << SBIT)

0.328125

In [342]:
bin(~0B1000)

TypeError: bin() takes exactly one argument (2 given)

In [32]:
Xinv

array([[-0.16666667,  0.5       , -0.33333333, -0.        ],
       [ 0.5       , -1.        , -0.5       ,  1.        ],
       [-0.5       ,  0.5       ,  1.        ,  0.        ],
       [ 0.16666667, -0.        , -0.16666667, -0.        ]])

## Resize for nearest neighbor

In [2]:
W, H, C = 400, 300, 3
im = np.arange( W * H * C, dtype=np.uint8).reshape(C, H, W).transpose(1, 2, 0)
rW, rH, rC = 123, 200, 3

In [9]:
def CLIP(val, lower, upper):
    val = lower if val < lower else val
    val = upper if val > upper else val
    return val

In [10]:
def resizeNN(im, w, h):
    ih, iw, ic = im.shape
    w_r = iw/w; h_r = ih/h
    img = np.empty((h, w, ic), dtype=np.uint8)
    p =0
    for i in range(h):
        hi = int(i*h_r + 0.5)
        for j in range(w):
            wi = int(j*w_r + 0.5)
            for k in range(ic):
                img[i,j,k] = im[hi, wi, k]
    return img

def resizeANN(im, w, h):
    ih, iw, ic = im.shape
    w_r = CR(iw,w); h_r = CR(ih,h)
    img = np.empty((h, w, ic), dtype=np.uint8)
    p =0
    for i in range(h):
        hi = (i * h_r + HBIT) >> FBIT
        for j in range(w):
            wi = (j * w_r + HBIT) >> FBIT 
            for k in range(ic):
                img[i,j,k] = im[hi, wi, k]
    return img



In [3]:
dog = Image.open("Dog.jpg")
dog.show()

In [24]:
rdog = resizeANN(np.array(dog), rH, rW)
im = Image.fromarray(rdog)
im.show()

In [25]:
## Time benchmark for floating point calc and fixed point calc

In [5]:
import time as T

RUN = 50
W,H = 224, 224
ti = T.time()
for i in range(RUN):
    rdog = resizeANN(np.array(dog), W, H)
toc1 = T.time() - ti
ti = T.time()
for i in range(RUN):
    rdog = resizeNN(np.array(dog), W, H)
toc2 = T.time() - ti
ti = T.time()
for i in range(RUN):
    rdog = np.asarray(cresizeANN(np.array(dog), W, H))
toc3 = T.time() - ti
print("toc1 vs toc2 vs toc3", toc1, toc2, toc3)

toc1 vs toc2 vs toc3 4.054576635360718 4.220536708831787 0.16736936569213867


In [ ]:
"""
https://stackoverflow.com/questions/12729228/simple-efficient-bilinear-interpolation-of-images-in-numpy-and-python
"""

## resize bilinear interpolation

In [7]:
def resizeBL(im, w, h):
    im_h, im_w, im_c = im.shape
    h_r = im_h / h
    w_r = im_w / w
    img = np.empty((h,w,im_c))
    
    for j in range(h):
        h0 = j * h_r
        n_h0 = int(h0)
        n_h1 = n_h0 + 1
        n_h1 = np.clip(n_h1, 0, im_h-1)
        h0 = h0 % 1
        h0V = 1-h0
        for k in range(w):
            w0 = k * w_r
            n_w0 = int(w0)
            n_w1 = n_w0 + 1
            n_w1 = np.clip(n_w1, 0, im_w-1)
            w0 = w0 % 1
            w0V = 1 - w0
            for i in range(im_c):
                #pix00 = im[n_h0, n_w0, i]
                #pix01 = im[n_h0, n_w1, i]
                #pix10 = im[n_h1, n_w0, i]
                #pix11 = im[n_h1, n_w1, i]
                #print("w0:%.2f, h0:%.2f, n_h0:%d, n_w0:%d, h_r:%.2f, w_r:%.2f" % (w0, h0, n_h0, n_w0, h_r, w_r))
                row_pix0 = im[n_h0, n_w0, i] * w0 + im[n_h0, n_w1, i] * w0V               
                row_pix1 = im[n_h1, n_w0, i] * w0 + im[n_h1, n_w1, i] * w0V

                img[j, k, i] = int(row_pix0 * h0 + row_pix1 * h0V)
                img[j, k, i] = CLIP(img[j, k, i], 0 ,255)

    return img.astype(np.uint8)             

def resizeABL(im, w, h):
    im_h, im_w, im_c = im.shape
    h_r = CR(im_h, h)
    w_r = CR(im_w, w)
    img = np.empty((h,w,im_c))

    for j in range(h):
        h0 = j * h_r
        n_h0 = h0 >> FBIT 
        n_h1 = n_h0 + 1
        #n_h1 = CLIP(n_h1, 0, im_h-1)
        h0 = (h0 + HBIT) % VBIT
        h0V = VBIT-h0
        for k in range(w):
            w0 = k * w_r
            n_w0 = w0 >> FBIT 
            n_w1 = n_w0 + 1
            #n_w1 = CLIP(n_w1, 0, im_w-1)
            w0 = (w0 + HBIT) % VBIT
            w0V = VBIT-w0
            for i in range(im_c):
                #pix00 = im[n_h0, n_w0, i]
                #pix01 = im[n_h0, n_w1, i]
                #pix10 = im[n_h1, n_w0, i]
                #pix11 = im[n_h1, n_w1, i]
                row_pix0 = ((im[n_h0, n_w0, i] * w0 + im[n_h0, n_w1, i] * w0V) + HBIT) >> FBIT
                row_pix1 = ((im[n_h1, n_w0, i] * w0 + im[n_h1, n_w1, i] * w0V) + HBIT) >> FBIT

                img[j, k, i] = ((row_pix0 * h0 + row_pix1 * h0V) + HBIT) >> FBIT
                #img[j, k, i] = CLIP(img[j, k, i], 0, 255)
    return img.astype(np.uint8)             

In [179]:
dog = Image.open("Dog.jpg")
dog_array = np.array(dog)


In [80]:
dog_bi = resizeBL(dog_array, 200, 144)

In [81]:
ndog = Image.fromarray(dog_bi)
ndog.show()

In [82]:
dog_bi = resizeABL(dog_array, 200, 144)

In [83]:
ndog = Image.fromarray(dog_bi)
ndog.show()

In [13]:
import time as T

RUN = 10
W,H = 224, 224
ti = T.time()
for i in range(RUN):
    rdog = resizeBL(np.array(dog), W, H)
toc1 = T.time() - ti
ti = T.time()
for i in range(RUN):
    rdog = resizeABL(np.array(dog), W, H)
toc2 = T.time() - ti
ti = T.time()
for i in range(RUN):
    rdog = cresizeABL(np.array(dog), W, H)
toc3 = T.time() - ti

print("toc1 vs toc2 vs toc3", toc1, toc2, toc3)


toc1 vs toc2 vs toc3 20.28959059715271 16.119190454483032 0.04034757614135742


In [22]:
## Bicubic

In [ ]:
def bicubic_interpolation2(xi, yi, zi, xnew, ynew):
    """
    @brief 
    Sample code:
    https://stackoverflow.com/questions/52700878/bicubic-interpolation-python
    
    BiCubic Interpolation explain:
    http://www.ahinson.com/algorithms_general/Sections/InterpolationRegression/InterpolationBicubic.pdf
    @param xi = old x data range
    @param yi = old y range
    @param zi = old values at grids points (x,y)
    @param xnew = new horizontal data ranges
    @param ynew = new horizontal data ranges. All inputs are 1D numpy arrays except zi which is 2D numpy array.
    @retval z = resize 2d array with shape = (xnew, ynew)
    """
    # check sorting
    if np.any(np.diff(xi) < 0) and np.any(np.diff(yi) < 0) and\
    np.any(np.diff(xnew) < 0) and np.any(np.diff(ynew) < 0):
        raise ValueError('data are not sorted')

    if zi.shape != (xi.size, yi.size):
        raise ValueError('zi is not set properly use np.meshgrid(xi, yi)')

    z = np.zeros((xnew.size, ynew.size))

    deltax = xi[1] - xi[0]
    deltay = yi[1] - yi[0] 
    for n, x in enumerate(xnew):
        for m, y in enumerate(ynew):

            if xi.min() <= x <= xi.max() and yi.min() <= y <= yi.max():

                i = np.searchsorted(xi, x) - 1
                j = np.searchsorted(yi, y) - 1

                x1  = xi[i]
                x2  = xi[i+1]

                y1  = yi[j]
                y2  = yi[j+1]

                px = (x-x1)/(x2-x1)
                py = (y-y1)/(y2-y1)

                f00 = zi[i-1, j-1]      #row0 col0 >> x0,y0
                f01 = zi[i-1, j]        #row0 col1 >> x1,y0
                f02 = zi[i-1, j+1]      #row0 col2 >> x2,y0

                f10 = zi[i, j-1]        #row1 col0 >> x0,y1
                f11 = zi[i, j]    #row1 col1 >> x1,y1
                f12 = zi[i, j+1]  #row1 col2 >> x2,y1

                f20 = zi[i+1,j-1]       #row2 col0 >> x0,y2
                f21 = zi[i+1,j]   #row2 col1 >> x1,y2
                f22 = zi[i+1,j+1] #row2 col2 >> x2,y2

                if 0 < i < xi.size-2 and 0 < j < yi.size-2:

                    f03 = zi[i-1, j+2]      #row0 col3 >> x3,y0

                    f13 = zi[i,j+2]         #row1 col3 >> x3,y1

                    f23 = zi[i+1,j+2]       #row2 col3 >> x3,y2

                    f30 = zi[i+2,j-1]       #row3 col0 >> x0,y3
                    f31 = zi[i+2,j]         #row3 col1 >> x1,y3
                    f32 = zi[i+2,j+1]       #row3 col2 >> x2,y3
                    f33 = zi[i+2,j+2]       #row3 col3 >> x3,y3

                elif i<=0: 

                    f03 = f02               #row0 col3 >> x3,y0

                    f13 = f12               #row1 col3 >> x3,y1

                    f23 = f22               #row2 col3 >> x3,y2

                    f30 = zi[i+2,j-1]       #row3 col0 >> x0,y3
                    f31 = zi[i+2,j]         #row3 col1 >> x1,y3
                    f32 = zi[i+2,j+1]       #row3 col2 >> x2,y3
                    f33 = f32               #row3 col3 >> x3,y3             

                elif j<=0:

                    f03 = zi[i-1, j+2]      #row0 col3 >> x3,y0

                    f13 = zi[i,j+2]         #row1 col3 >> x3,y1

                    f23 = zi[i+1,j+2]       #row2 col3 >> x3,y2

                    f30 = f20               #row3 col0 >> x0,y3
                    f31 = f21               #row3 col1 >> x1,y3
                    f32 = f22               #row3 col2 >> x2,y3
                    f33 = f23               #row3 col3 >> x3,y3


                elif i == xi.size-2 or j == yi.size-2:

                    f03 = f02               #row0 col3 >> x3,y0

                    f13 = f12               #row1 col3 >> x3,y1

                    f23 = f22               #row2 col3 >> x3,y2

                    f30 = f20               #row3 col0 >> x0,y3
                    f31 = f21               #row3 col1 >> x1,y3
                    f32 = f22               #row3 col2 >> x2,y3
                    f33 = f23               #row3 col3 >> x3,y3

                Z = np.array([f00, f01, f02, f03,
                             f10, f11, f12, f13,
                             f20, f21, f22, f23,
                             f30, f31, f32, f33]).reshape(4,4).transpose()

                X = np.tile(np.array([-1, 0, 1, 2]), (4,1))
                X[0,:] = X[0,:]**3
                X[1,:] = X[1,:]**2
                X[-1,:] = 1

                Cr = Z@np.linalg.inv(X)
                R = Cr@np.array([px**3, px**2, px, 1])

                Y = np.tile(np.array([-1, 0, 1, 2]), (4,1)).transpose()
                Y[:,0] = Y[:,0]**3
                Y[:,1] = Y[:,1]**2
                Y[:,-1] = 1

                Cc = np.linalg.inv(Y)@R

                z[n,m]=(Cc@np.array([py**3, py**2, py, 1]))


    return z

In [46]:
a = 0b100; b = 0b100

In [80]:
RUN = 10000000
res = -1
import time as T
ti = T.time()
a = 0

for i in range(RUN):
    a = 2
    a -= a + 1
toc1 = T.time() - ti
ti = T.time()
for i in range(RUN):
    a = 2
    a = a - a + 1
toc2 = T.time() - ti

print(toc1, toc2)

3.197888135910034 3.289870500564575


In [55]:
bin(a|b)

'0b110'

In [70]:
a = np.array([[1,2],[2,2]])
type(a)

numpy.ndarray

In [71]:
a.shape

(2, 2)

In [72]:
a.shape -1

TypeError: unsupported operand type(s) for -: 'tuple' and 'int'

In [332]:
print(bin(0b111 and 0b100))
print(bin(-(0b111 and ~(0b1 << 1))))

0b100
0b11


In [334]:
a = 1323192 
print(a and 0b11111110000000)))
print((a >> FBIT) << FBIT)

-8193
1318912


In [314]:
def ccCR(a,b):
    y = CR(12,7)
    Y = y >> FBIT << FBIT
    fy = y - Y
def cyCR(a,b):
    y = CR(12,7)
    Y = y and 
    fy = y - Y

In [315]:
y, Y, fy

(14043, 8192, 5851)

In [317]:
12/7

1.7142857142857142

In [316]:
fy / VBIT

0.7142333984375

In [313]:
y, Y

(14043, 2)

In [311]:
Y

4

In [307]:
def resizeBC(im, h, w):
    """
    @brief 
    Sample code:
    https://stackoverflow.com/questions/52700878/bicubic-interpolation-python
    
    BiCubic Interpolation explain:
    http://www.ahinson.com/algorithms_general/Sections/InterpolationRegression/InterpolationBicubic.pdf
    @param im = src image
    @param h = dst height
    @param w = dst width
    @retval img = resize 2d array with shape = (h, w)
    """
    ih, iw, ic = im.shape
    ih_, iw_ = ih-1, iw-1
    img = np.empty((h, w, ic),dtype=np.uint8)
    h_r = ih / h
    w_r = iw / w
    deltax = 1 #xi[1] - xi[0]
    deltay = 1 #yi[1] - yi[0]
    
    f = np.empty((4,4), np.uint8)
    met = clip
    
    for i in range(ic):
        for j in range(h):
            for k in range(w):

                #if xi.min() <= x <= xi.max() and yi.min() <= y <= yi.max():
                
                # common 
                x = k * w_r
                y = j * h_r
                W = int(x)
                H = int(y)
                
                # pad zero for out of bound value
                px = x - W
                py = y - H
                
                #px = (x-x1)/(x2-x1)
                #py = (y-y1)/(y2-y1)

                #f00 = zi[i-1, j-1]      #row0 col0 >> x0,y0
                #f01 = zi[i-1, j]        #row0 col1 >> x1,y0
                #f02 = zi[i-1, j+1]      #row0 col2 >> x2,y0
                """
                if (H == 0) and (W == 0):
                    f[0,0] = im[H  , W  , i]
                elif (W == 0):
                    f[0,0] = im[H-1  , W  , i]
                    f[1,0] = im[H    , W  , i]
                    f[2,0] = im[H+1  , W  , i]
                    f[3,0] = im[H+2  , W  , i]
                elif (H == 0):
                    f[0,0] = im[H  , W-1, i]      #row0 col0 >> x0,y0
                    f[0,1] = im[H  , W  , i]        #row0 col1 >> x1,y0
                    f[0,2] = im[H  , W+1, i]      #row0 col2 >> x2,y0
                    f[0,3] = im[H  , W+2, i]
                else:
                    f[0,0] = im[H-1, W-1, i]      #row0 col0 >> x0,y0
                    f[0,1] = im[H-1, W  , i]        #row0 col1 >> x1,y0
                    f[0,2] = im[H-1, W+1, i]      #row0 col2 >> x2,y0
                    f[0,3] = im[H-1, W+2, i]
                """
                    
                #f10 = zi[i, j-1]        #row1 col0 >> x0,y1
                #f11 = p00 = zi[i, j]    #row1 col1 >> x1,y1
                #f12 = p01 = zi[i, j+1]  #row1 col2 >> x2,y1
                d0w = met(W-1, iw_, 0)
                d1w = met(W  , iw_, 0)
                d2w = met(W+1, iw_, 0)
                d3w = met(W+2, iw_, 0)   
                #print((j,k,i),d0w,d1w,d2w,d3w,(ih,iw,ic),'size_r',(h_r,w_r),(j*h_r,k*w_r),met(W-1, iw_, 0))
                for q in range(4):
                    d0h = met(H+q-1, ih_, 0)
                    f[q,0] = im[d0h, d0w, i]      
                    f[q,1] = im[d0h, d1w, i]       
                    f[q,2] = im[d0h, d2w, i]     
                    f[q,3] = im[d0h, d3w, i] 
                f = f
                #print(f.shape,XINV.shape, XINV, type(f), type(XINV), f.dtype, XINV.dtype)
                Cr = f@XINV
                PPX = np.array([px*px*px, px*px, px, 1])
                #print('px:',px,'\n',PPX,'\n')
                #print("CR PPX",Cr.shape, PPX.shape, type(Cr), type(PPX), Cr.dtype, PPX.dtype)
                R  = Cr@PPX
                Cc = YINV@R

                img[j,k,i]= clip(int(Cc@(np.array([py*py*py, py*py, py, 1]))),255,0)


    return img
def resizeABC(im, h, w):
    """
    @brief 
    Sample code:
    https://stackoverflow.com/questions/52700878/bicubic-interpolation-python
    
    BiCubic Interpolation explain:
    http://www.ahinson.com/algorithms_general/Sections/InterpolationRegression/InterpolationBicubic.pdf
    @param im = src image
    @param h = dst height
    @param w = dst width
    @retval img = resize 2d array with shape = (h, w)
    """
    ih, iw, ic = im.shape
    ih_, iw_ = ih-1, iw-1
    img = np.empty((h, w, ic),dtype=np.uint8)
    F = np.empty((4,4), dtype=np.uint8)
    h_r = SCR(ih, h)
    w_r = SCR(iw, w)
   
    met = clip
    
    for j in range(h):
        y = j * h_r
        H = (y + hSBIT)>> SBIT << SBIT
        py = y - H
        PY = [SDR(SDR(py * py) * py), SDR(py*py), py, 1]
        for k in range(w):
            x = k * w_r
            W = int(x)              
            px = x - W
            d0w = met(W-1, iw_, 0)
            d1w = met(W  , iw_, 0)
            d2w = met(W+1, iw_, 0)
            d3w = met(W+2, iw_, 0) 
            for i in range(ic):  
                for q in range(4):
                    d0h = met(H+q-1, ih_, 0)
                    F[q,0] = im[d0h, d0w, i]      
                    F[q,1] = im[d0h, d1w, i]       
                    F[q,2] = im[d0h, d2w, i]     
                    F[q,3] = im[d0h, d3w, i] 
                
                Cr = F@XINV
                PPX [SDR(SDR(px * px) * px), SDR(px*px), py, 1]
                #print('px:',px,'\n',PPX,'\n')
                #print("CR PPX",Cr.shape, PPX.shape, type(Cr), type(PPX), Cr.dtype, PPX.dtype)
                R  = Cr@PPX
                Cc = YINV@R

                img[j,k,i]= clip(int(Cc@(np.array([py*py*py, py*py, py, 1]))),255,0)


    return img


In [308]:
W,H = 224,224
RUN = 1
ti = T.time()
for i in range(RUN):
    rdog = resizeBC(dog_array, W, H)
toc1 = T.time() - ti
ti = T.time()
for i in range(RUN):
     rdog = resizeABC(dog_array, W, H)
toc2 = T.time() - ti
print(toc1, toc2)

2.5745046138763428 2.117323875427246


In [299]:
imb = Image.fromarray(b.astype(np.uint8))
imb.show()

In [295]:
b = cresizeABL(dog_array,300,300)

In [296]:
imb = Image.fromarray(np.asarray(b))
imb.show()

In [214]:
ima = Image.fromarray(a)
imb = Image.fromarray(b.astype(np.uint8))

In [177]:
ima.show()

In [206]:
imb.show()

In [161]:
print(b.astype(np.uint8).shape)

(24, 26, 1)


In [126]:
a.dtype

dtype('uint8')

In [94]:
a.shape

(30, 40, 1)

In [372]:
X = np.tile(np.array([-1, 0, 1, 2]), (4,1))
X[0,:] = X[0,:]**3
X[1,:] = X[1,:]**2
X[-1,:] = 1

a = np.linalg.inv(X)
a * 6

array([[-1.,  3., -2., -0.],
       [ 3., -6., -3.,  6.],
       [-3.,  3.,  6.,  0.],
       [ 1., -0., -1., -0.]])

In [35]:
Y = np.tile(np.array([-1, 0, 1, 2]), (4,1)).transpose()
Y[:,0] = Y[:,0]**3
Y[:,1] = Y[:,1]**2
Y[:,-1] = 1

Cc = np.linalg.inv(Y)

In [43]:
cy = Cc * 6
cy[1,3] = 0

In [44]:
cy

array([[-1.,  3., -3.,  1.],
       [ 3., -6.,  3.,  0.],
       [-2., -3.,  6., -1.],
       [ 0.,  6.,  0.,  0.]])

In [30]:
a * 6

array([[-1.,  3., -2., -0.],
       [ 3., -6., -3.,  6.],
       [-3.,  3.,  6.,  0.],
       [ 1., -0., -1., -0.]])

In [15]:
import cv2

In [21]:
RUN = 300
W,H = 224, 224
ti = T.time()
for i in range(RUN):
    rdog = cv2.resize(np.array(dog), (W, H), interpolation=cv2.INTER_NEAREST)
toc1 = T.time() - ti
ti = T.time()
for i in range(RUN):
    rdog = cv2.resize(np.array(dog), (W, H), interpolation=cv2.INTER_LINEAR)
toc2 = T.time() - ti
ti = T.time()
for i in range(RUN):
    rdog = cv2.resize(np.array(dog), (W, H), interpolation=cv2.INTER_AREA)
toc3 = T.time() - ti
ti = T.time()
for i in range(RUN):
    rdog = cv2.resize(np.array(dog), (W, H), interpolation=cv2.INTER_CUBIC)
toc4 = T.time() - ti
ti = T.time()
for i in range(RUN):
    rdog = cv2.resize(np.array(dog), (W, H), interpolation=cv2.INTER_LANCZOS4)
toc5 = T.time() - ti
ti = T.time()
for i in range(RUN):
    rdog = np.asarray(cresizeANN(np.array(dog), H, W))
toc6 = T.time() - ti
ti = T.time()
for i in range(RUN):
    rdog = np.asarray(cresizeABL(np.array(dog), H, W))
toc7 = T.time() - ti

print("toc1:%.4f\ntoc2:%.4f\ntoc3:%.4f\ntoc4:%.4f\ntoc5:%f" %( toc1, toc2, toc3, toc4, toc5))
print("toc6c:%.4f\ntoc7c:%.4f" % (toc6, toc7))

toc1:0.8641
toc2:0.9948
toc3:2.0058
toc4:1.0666
toc5:1.810819
toc6c:0.9856
toc7c:1.0837
